<a href="https://colab.research.google.com/github/jackiekuen2/notes-handson-ml-tf/blob/master/ch9_TensorFlow_MinibatchGradientDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# a = []
# while(1):
#     a.append("1")

In [0]:
%tensorflow_version 1.x

import tensorflow as tf

print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

## Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
m, n = scaled_housing_data.shape

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [0]:
print(m, n)
print(scaled_housing_data_plus_bias.shape)

20640 8
(20640, 9)


# Mini-batch Gradient Descent

In [0]:
n_epochs = 10 #1000
learning_rate = 0.01

# Change to placeholder nodes
# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

# Randomly initialize a tensor theta, tf.random_uniform() similar to np.rand()
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y

# Mean Square Error, tf.reduce_mean similar to np.mean()
mse = tf.reduce_mean(tf.square(error), name='mse')

# Using GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# Minimize MSE
training_op = optimizer.minimize(mse)

# Initialize global variables
init = tf.global_variables_initializer()

# Define batch size
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

# Fetch the mini-batches one by one
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) # Set a different random seed every epoch, so that it fetches different indices for mini-batches
    indices = np.random.randint(m, size=batch_size) # Randomly pick 100 indices
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()

print("Best theta: ", best_theta)

Best theta:  [[ 2.070205  ]
 [ 0.8535242 ]
 [ 0.12390032]
 [-0.28697598]
 [ 0.36043227]
 [ 0.00520482]
 [-0.01296346]
 [-0.83276993]
 [-0.8003151 ]]
